Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Смотрим на данные

In [8]:
df = pd.read_csv("2015.csv")

In [9]:
df.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [10]:
df.describe()

,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000
mean,79.493671,5.375734,0.047885,0.846137,0.991046,0.630259,0.428615,0.143422,0.237296,2.098977
std,45.754363,1.145010,0.017146,0.403121,0.272369,0.247078,0.150693,0.120034,0.126685,0.553550
min,1.000000,2.839000,0.018480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328580
25%,40.250000,4.526000,0.037268,0.545808,0.856823,0.439185,0.328330,0.061675,0.150553,1.759410
50%,79.500000,5.232500,0.043940,0.910245,1.029510,0.696705,0.435515,0.107220,0.216130,2.095415
75%,118.750000,6.243750,0.052300,1.158448,1.214405,0.811013,0.549092,0.180255,0.309883,2.462415
max,158.000000,7.587000,0.136930,1.690420,1.402230,1.025250,0.669730,0.551910,0.795880,3.602140


Избавимся от ненужных столбцов.

In [29]:
df.Country.value_counts().sort_values()

Argentina                  1
Uzbekistan                 1
Austria                    1
Ecuador                    1
Venezuela                  1
Trinidad and Tobago        1
Tajikistan                 1
Mauritania                 1
Laos                       1
Indonesia                  1
Azerbaijan                 1
Congo (Brazzaville)        1
Colombia                   1
Bahrain                    1
Jamaica                    1
Morocco                    1
Guatemala                  1
Sweden                     1
Australia                  1
Mexico                     1
Madagascar                 1
Afghanistan                1
Bulgaria                   1
Croatia                    1
Belarus                    1
Senegal                    1
Chile                      1
Italy                      1
Sri Lanka                  1
Thailand                   1
                          ..
Hungary                    1
Guinea                     1
Brazil                     1
Iran          

Видим, что каждая страна представлена 1 раз, поэтому это больше тянет на индекс, чем на признак, выбрасываем.

In [30]:
df.corr()

,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
Happiness Rank,1.000000,-0.992105,0.158516,-0.785267,-0.733644,-0.735613,-0.556886,-0.372315,-0.160142,-0.521999
Happiness Score,-0.992105,1.000000,-0.177254,0.780966,0.740605,0.724200,0.568211,0.395199,0.180319,0.530474
Standard Error,0.158516,-0.177254,1.000000,-0.217651,-0.120728,-0.310287,-0.129773,-0.178325,-0.088439,0.083981
Economy (GDP per Capita),-0.785267,0.780966,-0.217651,1.000000,0.645299,0.816478,0.370300,0.307885,-0.010465,0.040059
Family,-0.733644,0.740605,-0.120728,0.645299,1.000000,0.531104,0.441518,0.205605,0.087513,0.148117
Health (Life Expectancy),-0.735613,0.724200,-0.310287,0.816478,0.531104,1.000000,0.360477,0.248335,0.108335,0.018979
Freedom,-0.556886,0.568211,-0.129773,0.370300,0.441518,0.360477,1.000000,0.493524,0.373916,0.062783
Trust (Government Corruption),-0.372315,0.395199,-0.178325,0.307885,0.205605,0.248335,0.493524,1.000000,0.276123,-0.033105
Generosity,-0.160142,0.180319,-0.088439,-0.010465,0.087513,0.108335,0.373916,0.276123,1.000000,-0.101301
Dystopia Residual,-0.521999,0.530474,0.083981,0.040059,0.148117,0.018979,0.062783,-0.033105,-0.101301,1.000000


"Happiness Rank" сильно скореллировано с "Happiness Score".

In [11]:
dropped_happy = df.drop(["Standard Error", "Country", "Happiness Rank", "Region"], axis=1)
dropped_happy.head()

,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


Обучим несколько моделей и сравним результат.

In [12]:
X = dropped_happy.drop("Happiness Score", axis = 1)
y = dropped_happy['Happiness Score']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.3)

In [14]:
def evaluate(model):
    pred = model.predict(X_test)
    print("mse = ", mean_squared_error(y_test, pred))
    print("mae = ", mean_absolute_error(y_test, pred))
    print("r2  = ", r2_score(y_test, pred))

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)
evaluate(lr)

mse =  7.776885868236315e-08
mae =  0.00023368149287667875
r2  =  0.9999999415792218


In [18]:
coef = zip(X.columns, lr.coef_)
coef_df = pd.DataFrame(list(zip(X.columns, lr.coef_)), columns=['features', 'coefficients'])
coef_df

,features,coefficients
0,Economy (GDP per Capita),1.000083
1,Family,1.000011
2,Health (Life Expectancy),0.999966
3,Freedom,0.999771
4,Trust (Government Corruption),0.999969
5,Generosity,0.999953
6,Dystopia Residual,1.000002


In [21]:
tree = DecisionTreeRegressor()
param_grid = {'criterion': ['mse', 'mae'] , 'max_depth': [5, 10, 30, 100], 'min_samples_leaf': [1, 2, 5, 10]}
search = GridSearchCV(tree, n_jobs=-1, param_grid=param_grid, cv=5)
search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': [5, 10, 30, 100],
                         'min_samples_leaf': [1, 2, 5, 10]},
             pre_dispatch='2*n_jobs', refit

In [22]:
evaluate(search.best_estimator_)

mse =  0.16937749479166667
mae =  0.33771875
r2  =  0.8727618583324296


In [23]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
evaluate(rf)

mse =  0.15438674166666655
mae =  0.3197124999999999
r2  =  0.8840230685195825


/home/gulnara/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [27]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
evaluate(xgb)

mse =  0.08835095166739482
mae =  0.23830160101254774
r2  =  0.9336298430995938


Вывод: видим, что лучше всего себя показала линейная регрессия.